In [3]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.utils import get_file
import tarfile
import os
import numpy as np
import json
from collections import Counter
from keras.optimizers import SGD

In [5]:
path = get_file('pet_images', 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz')
tar = tarfile.open(path)
tar.extractall()
tar.close()
os.rename('images', 'pet_images')

In [6]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [7]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [8]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = Dense(1024, name='dense', activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15
 - 1717s - loss: 1.7371 - acc: 0.6495
Epoch 2/15
 - 1706s - loss: 0.5870 - acc: 0.8233
Epoch 3/15
 - 1706s - loss: 0.4028 - acc: 0.8708
Epoch 4/15
 - 1706s - loss: 0.3186 - acc: 0.8970
Epoch 5/15


In [6]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=2
)

Epoch 1/10
34s - loss: 0.0207 - acc: 0.9935
Epoch 2/10
34s - loss: 0.0184 - acc: 0.9946
Epoch 3/10
34s - loss: 0.0176 - acc: 0.9955
Epoch 4/10
34s - loss: 0.0166 - acc: 0.9959
Epoch 5/10
34s - loss: 0.0155 - acc: 0.9972
Epoch 6/10
34s - loss: 0.0156 - acc: 0.9972
Epoch 7/10
34s - loss: 0.0149 - acc: 0.9969
Epoch 8/10
34s - loss: 0.0152 - acc: 0.9965
Epoch 9/10
34s - loss: 0.0151 - acc: 0.9963
Epoch 10/10
35s - loss: 0.0146 - acc: 0.9976


In [11]:
json.dump(idx_to_labels, open('zoo/09.3 pet_labels.json', 'w'))
model.save('zoo/09.3 retrained pet recognizer.h5')
